In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import os
from keras.callbacks import EarlyStopping
from datasets import load_dataset, Dataset
import csv
import re
import tensorflow.keras.backend as K
from tensorflow.keras.layers  import Layer
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import constraints
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
#from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from tensorflow.keras.layers import Dense, Activation, Dropout, Embedding

from tensorflow.keras.layers import BatchNormalization

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
from keras.layers import Dense, Activation, Embedding, Input
from keras.models import Model

#Hyperparameters

In [ ]:
epochs = 5
batch_size = 32
margin = 1  # Margin for constrastive loss.
max_len = 1500

# Data Loading

In [ ]:
#Loading banking77 dataset
from datasets import load_dataset

dataset = load_dataset("PolyAI/banking77" )

# Data Preprocessing

In [ ]:
data = dataset['train']['text']
y = dataset['train']['label']

data_test = dataset['test']['text']
y_test = dataset['test']['label']


#convert list to pandas series
data = pd.Series(data)
y  = pd.Series(y)

data_test = pd.Series(data_test)
y_test  = pd.Series(y_test)

In [ ]:
#splitting train dataset to training and validation parts

xtrain, xvalid, ytrain, yvalid = train_test_split(data, y,
                                                  stratify=y,
                                                  random_state=42,
                                                  test_size=0.1, shuffle=True)


In [ ]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)
#xtest_seq  = token.texts_to_sequences(data_test)

#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)
#xtest_pad  = pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

# Model

In [ ]:
# Aspect modeling based on orginal paper "An Unsupervised Neural Attention Model for Aspect Extraction"

class Attention(Layer):
    def __init__(self, W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Content Attention mechanism.
        Supports Masking.
        """
        self.supports_masking = True
        self.init = initializers.glorot_uniform

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert type(input_shape) == list
        assert len(input_shape) == 2

        self.steps = input_shape[0][1]

        self.W = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),
                                    initializer=self.init,
                                    name='{}_W'.format(self.name),
                                    regularizer=self.W_regularizer,
                                    constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(1,),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        self.built = True

    def compute_mask(self, input_tensor, mask=None):
        return None

    def call(self, input_tensor, mask=None):
        x = input_tensor[0]
        y = input_tensor[1]
        #mask = mask[0]

        y = K.transpose(K.dot(self.W, K.transpose(y)))
        y = K.expand_dims(y, -2)
        y = K.repeat_elements(y, self.steps, axis=1)
        eij = K.sum(x*y, axis=-1)

        if self.bias:
            b = K.repeat_elements(self.b, self.steps, axis=0)
            eij += b

        eij = K.tanh(eij)
        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        return a

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1])

class WeightedSum(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(WeightedSum, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        assert type(input_tensor) == list
        #assert type(mask) == list   #this line has been changed by me

        x = input_tensor[0]
        a = input_tensor[1]

        a = K.expand_dims(a)
        weighted_input = x * a

        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][-1])

    def compute_mask(self, x, mask=None):
        return None

class WeightedAspectEmb(Layer):
    def __init__(self, input_dim, output_dim,
                 init='uniform', input_length=None,
                 W_regularizer=None, activity_regularizer=None,
                 W_constraint=None,
                 weights=None, dropout=0., **kwargs):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.init = initializers.get(init)
        self.input_length = input_length
        self.dropout = dropout

        self.W_constraint = constraints.get(W_constraint)
        self.W_regularizer = regularizers.get(W_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        if 0. < self.dropout < 1.:
            self.uses_learning_phase = True
        self.initial_weights = weights
        kwargs['input_shape'] = (self.input_length,)
        dtype = K.floatx()
        super(WeightedAspectEmb, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(shape=(self.input_dim, self.output_dim),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
        self.built = True

    def compute_mask(self, x, mask=None):
        return None

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def call(self, x, mask=None):
        return K.dot(x, self.W)


class Average(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
        return K.sum(x, axis=-2) / K.sum(mask, axis=-2)

    def compute_output_shape(self, input_shape):
        return input_shape[0:-2]+input_shape[-1:]

    def compute_mask(self, x, mask=None):
        return None


class MaxMargin(Layer):
    def __init__(self, **kwargs):
        super(MaxMargin, self).__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        z_s = input_tensor[0]
        z_n = input_tensor[1]
        r_s = input_tensor[2]

        z_s = z_s / K.cast(K.epsilon() + K.sqrt(K.sum(K.square(z_s), axis=-1, keepdims=True)), K.floatx())
        z_n = z_n / K.cast(K.epsilon() + K.sqrt(K.sum(K.square(z_n), axis=-1, keepdims=True)), K.floatx())
        r_s = r_s / K.cast(K.epsilon() + K.sqrt(K.sum(K.square(r_s), axis=-1, keepdims=True)), K.floatx())

        steps = z_n.shape[1]

        pos = K.sum(z_s*r_s, axis=-1, keepdims=True)

        pos = K.repeat_elements(pos, steps, axis=-1)
        r_s = K.expand_dims(r_s, -2)
        r_s = K.repeat_elements(r_s, steps, axis=1)
        neg = K.sum(z_n*r_s, axis=-1)

        loss = K.cast(K.sum(K.maximum(0., (1. - (pos + neg))), axis=-1, keepdims=True), K.floatx())
        return loss

    def compute_mask(self, input_tensor, mask=None):
        return None

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], 1)

# Aspect model with softmax head

In [ ]:
# creating the model with keras

input = layers.Input(shape=(1500,), dtype='int32')
word_emb = Embedding(len(word_index)  , 300 ,trainable=True, mask_zero=True, name='word_emb')
e_w = word_emb(input)

y_s = Average()(e_w)

att_weights = Attention(name='att_weights')([e_w, y_s])
z_s = WeightedSum()([e_w, att_weights])

output = layers.Dense(77)(z_s)

cls_model = keras.Model(inputs= input, outputs=output) # creating the model with out 0 or 1 (binary output)

# compiling the model
cls_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer="adam", metrics = ['accuracy'])
cls_model.summary()


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
history = cls_model.fit(
    xtrain_pad,
    ytrain,
    validation_data=(xvalid_pad, yvalid),
    batch_size=8,
    epochs=20,
    callbacks=[es]
)

# evaluate the model on the test dataset

cls_model.evaluate(xtest_pad, y_test, batch_size=batch_size)

# Aspect model with TF-idf

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# building tf-idf vectors for each sample in our dataset
tfv = TfidfVectorizer(min_df=1,  max_features=None,
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 1), use_idf=True,smooth_idf=1,sublinear_tf=1,
            #stop_words = 'english'
)

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)
xtest_tfv = tfv.transform(data_test)


xtrain_tfv = xtrain_tfv.toarray()
xvalid_tfv = xvalid_tfv.toarray()
xtest_tfv = xtest_tfv.toarray()


In [ ]:
# creating the model with keras

input1 = layers.Input(shape=(1500,), dtype='int32')
input2 = layers.Input(shape=(1129,), dtype='float32')
word_emb = Embedding(len(word_index) , 300,trainable=True, mask_zero=True, name='word_emb')
e_w = word_emb(input1)
y_s = Average()(e_w)
att_weights = Attention(name='att_weights')([e_w, y_s])
z_s = WeightedSum()([e_w, att_weights])
z_s = tf.concat(axis=1,values=[z_s, input2])

output = layers.Dense(77, activation='softmax')(z_s)
cls_model = keras.Model(inputs= [input1, input2], outputs=output)


#compile and fit the model
cls_model.compile(loss='SparseCategoricalCrossentropy', optimizer="adam", metrics=["accuracy"])
cls_model.summary()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
history = cls_model.fit(
    [xtrain_pad, xtrain_tfv],
    ytrain,
    validation_data=([xvalid_pad, xvalid_tfv], yvalid),
    batch_size=8,
    epochs=20,
    callbacks=[es]
)


In [ ]:
#evaluate the model on the test dataset
cls_model.evaluate([xtest_pad, xtest_tfv], y_test, batch_size=batch_size)

# Aspect with Bert

In [ ]:
# tokenization the input for the bert model
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

train_tokenized = tokenizer(list(xtrain),padding='max_length', truncation=True, max_length=128)
valid_tokenized = tokenizer(list(xvalid),padding='max_length', truncation=True, max_length=128)
test_tokenized = tokenizer(list(data_test),padding='max_length', truncation=True, max_length=128)

train_input_ids = np.array(train_tokenized['input_ids'])
valid_input_ids = np.array(valid_tokenized['input_ids'])
test_input_ids = np.array(test_tokenized['input_ids'])



train_mask = np.array(train_tokenized['attention_mask'])
valid_mask = np.array(valid_tokenized['attention_mask'])
test_mask = np.array(test_tokenized['attention_mask'])

In [ ]:
#loading the bert model
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)

In [ ]:
#creating the model with keras


input1 = layers.Input(shape=(128,), dtype='int32')


#attention_mask = K.not_equal(input1,  0 )
mask = K.not_equal(input1,  0 or 101 or 102 )
attention_mask = K.not_equal(input1,  0  )

hidden_states = bert_model(input1, attention_mask = attention_mask)
cls = hidden_states[1]
e_w = hidden_states[0][:,1:,:]


#y_s = Average()(e_w)
att_weights = Attention(name='att_weights')([e_w, cls] )
z_s = WeightedSum()([e_w, att_weights],  )
z_s = tf.concat(axis=1,values=[z_s, cls])
output = layers.Dense(77, activation="softmax")(z_s)








cls_model = keras.Model(inputs=input1, outputs=output) # creating the model with out 0 or 1 (binary output)

#compile and fit the model
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 3)
cls_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)
history = cls_model.fit(
    [train_input_ids],
    ytrain,
    validation_data=(valid_input_ids, yvalid),
    batch_size=32,
    epochs=5,
    callbacks = [es]
)

In [ ]:
#evaluate the model on the test datset
print("Evaluate on test data")
results = cls_model.evaluate([valid_input_ids, valid_mask], yvalid, batch_size=32)
print("test loss, test acc:", results)

# Bert with TF-idf


In [ ]:
#input tokenization for bert model

from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

train_tokenized = tokenizer(list(xtrain),padding='max_length', truncation=True, max_length=128)
valid_tokenized = tokenizer(list(xvalid),padding='max_length', truncation=True, max_length=128)
test_tokenized = tokenizer(list(data_test),padding='max_length', truncation=True, max_length=128)

train_input_ids = np.array(train_tokenized['input_ids'])
valid_input_ids = np.array(valid_tokenized['input_ids'])
test_input_ids = np.array(test_tokenized['input_ids'])



train_mask = np.array(train_tokenized['attention_mask'])
valid_mask = np.array(valid_tokenized['attention_mask'])
test_mask = np.array(test_tokenized['attention_mask'])

In [ ]:
#loading bert model
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

In [ ]:
#creating model with keras
input1 = layers.Input(shape=(128,), dtype='int32')
input2 = layers.Input(shape=(1899,), dtype='float32')
attention_mask = layers.Input(shape=(128,), dtype='int32')


##### Construct bert layer #####
hidden_states = bert_model(input1, attention_mask = attention_mask)[1] # using pooler output

cls = hidden_states
cls= tf.concat(axis=1,values=[cls, input2])
output = layers.Dense(77, activation="softmax")(cls)

cls_model = keras.Model(inputs= [input1, input2, attention_mask], outputs=output)


# compile and fot the model
cls_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)


s = EarlyStopping(monitor='val_loss', mode='min', verbose=5, patience = 3)

history = cls_model.fit(
    [train_input_ids, xtrain_tfv, train_mask],
    ytrain,
    validation_data=([valid_input_ids, xvalid_tfv, valid_mask], yvalid),
    batch_size=32,
    epochs=100,
    callbacks = [es]
)




In [ ]:
#evaluate the model on test dataset
cls_model.evaluate([test_input_ids, xtest_tfv, test_mask], y_test, batch_size=batch_size)